In [13]:
from bs4 import BeautifulSoup #temque instalar o BeautifulSoup
import urllib.request
import urllib.parse
import numpy as np

In [14]:
def get_file_names(ano):
    anos_bissextos = {"2008", "2012", "2016", "2020"}
    if ano in anos_bissextos:
        fevereiro_tem_29 = True
    else:
        fevereiro_tem_29 = False
    
    meses = {
        'janeiro' : '01',
        'fevereiro' : '02',
        'marco' : '03',
        'abril' : '04',
        'maio' : '05',
        'junho' : '06',
        'julho' : '07',
        'agosto' : '08',
        'setembro' : '09',
        'outubro' : '10',
        'novembro' : '11',
        'dezembro' : '12'
    }
    
    meses_com_31 = ["janeiro", "marco", "maio", "julho", "agosto", "outubro", "dezembro"]
    meses_com_30 = ["abril", "junho", "setembro", "novembro"]
    
    file_names = []
    for m in meses :
        if m == "fevereiro" and not fevereiro_tem_29:
            total_days = 28
        elif m == "fevereiro" and fevereiro_tem_29:
            total_days = 29
        elif m in meses_com_30:
            total_days = 30
        elif m in meses_com_31:
            total_days = 31
        
        #bases diarias
        for d in [str(d).zfill(2) for d in np.arange(start=1,stop=total_days+1)]:
            file_name = 'pass_transp_{year}_{month}_{day}.xls'.format(year=ano, month=meses[m], day=d)
            file_names.append(file_name)
            
        #agregado mensal
        file_name = 'pass_transp_agreg_{year}_{month}.xls'.format(year=ano, month=meses[m])
        file_names.append(file_name)
        
    return file_names

In [16]:
f = open('log.txt', 'w')

anos = [str(y) for y in np.arange(start=2013, stop=2020)]

links = ['https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/index.php?p=164379',
         'http://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/index.php?p=152417',
         'http://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/index.php?p=188767',
         'http://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/index.php?p=209427',
         'http://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/index.php?p=228269',
         'https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/index.php?p=247850',
         'https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=269652'
         ]
link_ano = dict(zip(anos, links))

for ano in anos:
    file_names = get_file_names(ano)
    if ano == "2013":
        file_names = file_names[-190:] #ano incompleto
    if ano == "2019":
        file_names = file_names[:120] #ano incompleto
    
    resp = urllib.request.urlopen(link_ano[ano])
    soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
    links = soup.find_all('a',href=True)
    
    links_tratados = []
    for link in links:
        if link['href'][-3:] == 'xls':
            links_tratados.append(link['href'])
        elif link['href'][-4:] == 'xlsx':
            links_tratados.append(link['href'])
            file_names[len(links_tratados) - 1] = file_names[len(links_tratados) - 1] + 'x'

    
    link_file_name = dict(zip(file_names, links_tratados))
    
    if ano == "2016":
        link_file_name['pass_transp_2016_10_21.xls'] = 'http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/acesso_a_informacao/2016/outubro/passageiros/Passag-20161021.xls'
    elif ano == "2017":
        link_file_name['pass_transp_2017_11_08.xls'] = 'http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/Passag-20171108.xls'
    
    
    for file_name in link_file_name.keys():
        try:
            print('Downloading {file_name}...'.format(file_name=file_name))
            urllib.request.urlretrieve(urllib.parse.quote(link_file_name[file_name],safe=':/'), './raw/{file_name}'.format(file_name=file_name))
            print(urllib.parse.quote(link_file_name[file_name],safe=':/'))
            f.write('{file_name} ,'.format(file_name=file_name))
            f.write(urllib.parse.quote(link_file_name[file_name],safe=':/') + "\n")
        except urllib.error.HTTPError:
            print("\tFalha no download")
            
f.close()

http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/julho/passageiros/Passag-Transportados-20130701.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/julho/passageiros/Passag-Transportados-20130702.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/julho/passageiros/Passag-Transportados-20130703.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/julho/passageiros/Passag-Transportados-20130704.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/julho/passageiros/Passag-Transportados-20130705.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/julho/passageiros/Passag-Transportados-20130706.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/julho/passageiros/Passag-Transportados-20130707.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/uplo

http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/agosto/passageiros/Passag-Transportados_20130816.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/agosto/passageiros/Passag-Transportados_20130817.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/agosto/passageiros/Passag-Transportados_20130818.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/agosto/passageiros/Passag-Transportados_20130819.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/agosto/passageiros/Passag-Transportados_20130820.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/agosto/passageiros/Passag-Transportados_20130821.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/agosto/passageiros/Passag-Transportados_20130822.xls
http://www.prefeitura.sp.gov.br/cidade/secretari

http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/outubro/passageiros/Passag-20131002.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/outubro/passageiros/Passag-20131003.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/outubro/passageiros/Passag-20131004.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/outubro/passageiros/Passag-20131005.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/outubro/passageiros/Passag-20131006.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/outubro/passageiros/Passag-20131007.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/outubro/passageiros/Passag-20131008.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/outubro/passageiros/Passag-20131009.xls
http://www.prefe

http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/novembro/passageiros/Passag-20131120.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/novembro/passageiros/Passag-20131121.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/novembro/passageiros/Passag-20131122.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/novembro/passageiros/Passag-20131123.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/novembro/passageiros/Passag-20131124.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/novembro/passageiros/Passag-20131125.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/novembro/passageiros/Passag-20131126.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2013/novembro/passageiros/Passag-20131127.xls
http://w

http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2014/janeiro/passageiros/Passag-20140107.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2014/janeiro/passageiros/Passag-20140108.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2014/janeiro/passageiros/Passag-20140109.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2014/janeiro/passageiros/Passag-20140110.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2014/janeiro/passageiros/Passag-20140111.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2014/janeiro/passageiros/Passag-20140112.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2014/janeiro/passageiros/Passag-20140113.xls
http://www.prefeitura.sp.gov.br/cidade/secretarias/upload/transportes/SPTrans/2014/janeiro/passageiros/Passag-20140114.xls
http://www.prefe

...

https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/020818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/030818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/040818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/050818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/060818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/070818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/080818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/090818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/100818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/110818%281%29.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/120818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/130818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/140818.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/150818.xls
https://www.p

https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/07102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/08102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/09102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/10102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/11102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/12102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/13102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/14102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/15102018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secret

https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/Consolidado%20Novembro%202018.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/01122018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/02122018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/03122018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/04122018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/05122018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/06122018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/07122018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/08122018_Dados%20para%20Transpar%C3%AAncia.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/

https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/27-JAN.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/28-JAN.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/29-JAN.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/30-JAN.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/31-JAN.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/Consolidado%20Janeiro%202019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/01-FEV.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/02-FEV.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/03-FEV.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/04-FEV.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/05-FEV.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/06-FEV.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/07-FEV.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/08-FEV.xl

https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/06ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/07ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/08ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/09ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/10ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/11ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/12ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/13ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/14ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/15ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/16ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/17ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/18ABR2019.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias